In [1]:
import os
os.chdir('../')

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str 
    alpha: float
    l1_ratio: float
    target_column: str

In [3]:
from src.datascience.constants import *
from src.datascience.utils.common import *

In [8]:
class ConfigurationManager:
    def __init__(self,
                 config_file=CONFIG_FILE_PATH,
                 params_file=PARAMS_FILE_PATH,
                 schema_file=SCHEMA_FILE_PATH
                 ):
        self.config = read_yaml(config_file)
        self.params = read_yaml(params_file)
        self.schema = read_yaml(schema_file)
        create_directories([self.config.artifacts_root])
        
    def get_model_trainer_config(self)-> ModelTrainerConfig:
        config = self.config.model_trainer
        schema = self.schema.TARGET_COLUMN
        params = self.params.ElasticNet
        create_directories([config.root_dir])
        
        model_trainer_config = ModelTrainerConfig(
            root_dir = config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name, 
            alpha = params.alpha,
            l1_ratio = params.l1_ratio,
            target_column = schema.name
        )
        
        return model_trainer_config

In [5]:
import pandas as pd
from src.datascience.utils.common import logger
from sklearn.linear_model import ElasticNet
import os
import joblib

In [6]:
class Modeltrainer:
    def __init__(self,config:ModelTrainerConfig):
        self.config = config
    
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)
        
        train_x = train_data.drop([self.config.target_column],axis=1)
        test_x = test_data.drop([self.config.target_column],axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]
        
        lr = ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio, random_state=42)
        lr.fit(train_x,train_y)
        
        joblib.dump(lr,os.path.join(self.config.root_dir,self.config.model_name))
        

In [9]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = Modeltrainer(model_trainer_config)
    model_trainer.train()
except Exception as err:
    raise err

[2025-05-26 18:26:19,783]: INFO: common: yaml file: config/config.yaml loaded successfully.]
[2025-05-26 18:26:19,784]: INFO: common: yaml file: params.yaml loaded successfully.]
[2025-05-26 18:26:19,786]: INFO: common: yaml file: schema.yaml loaded successfully.]
[2025-05-26 18:26:19,787]: INFO: common: Created directory at: artifacts]
[2025-05-26 18:26:19,787]: INFO: common: Created directory at: artifacts/model_trainer]
